In [1]:
# Install libreries

#!pip install python-dotenv openai anthropic gradio

In [2]:
# Imports

from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import gradio as gr
import os

In [3]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')

In [4]:
openai = OpenAI()
claude = anthropic.Anthropic()
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"

In [51]:
system_prompt = "You are an intelligent assistant specializing in software documentation and code understanding.\n"
system_prompt += "Your task is to analyze given Python code snippets, understand their functionality, and produce detailed and accurate docstrings and inline comments.\n"
system_prompt += "Follow these guidelines:\n\n"
system_prompt += "1. Use triple double quotes (\"\"\") for Python docstrings.\n"
system_prompt += "2. Follow the Google-style docstring format, including:\n"
system_prompt += "   - A concise description of the function or class.\n"
system_prompt += "   - Args: A list describing all parameters with types and purposes.\n"
system_prompt += "   - Returns: A description of the return value, including the type.\n"
system_prompt += "   - Example: An example of the function and the output.\n\n"
system_prompt += "3. Add inline comments for non-obvious logic or important steps in the code.\n"
system_prompt += "   - Inline comments should be concise and informative.\n\n"
system_prompt += "4. Ensure all docstrings and comments adhere to Python's PEP 8 guidelines.\n"
system_prompt += "5. Use clear and professional language, avoiding over-explaining obvious code.\n\n"
system_prompt += "Example Input:\n"
system_prompt += "def add_numbers(a, b):\n"
system_prompt += "    return a + b\n\n"
system_prompt += "Example Output:\n"
system_prompt += "def add_numbers(a, b):\n"
system_prompt += "    \"\"\"Adds two numbers and returns the result.\n\n"
system_prompt += "    Args:\n"
system_prompt += "        a (int): The first number.\n"
system_prompt += "        b (int): The second number.\n\n"
system_prompt += "    Returns:\n"
system_prompt += "        int: The sum of the two numbers.\n"
system_prompt += "    Example:\n"
system_prompt += "        >>>add_numbers(1, 2):\n"
system_prompt += "        '3'\n"
system_prompt += "    \"\"\"\n"
system_prompt += "    return a + b  # Add the two numbers and return the result\n"

In [25]:
def user_prompt_for(python_code):
    user_prompt = "Analyze the following Python code snippet and add detailed docstrings and inline comments to improve readability and maintainability. "
    user_prompt += "Ensure the docstrings follow the Google-style format with sections such as Description, Args, Returns, and Raises if applicable. "
    user_prompt += "Respond only with the updated Python code, including the added docstrings and inline comments, and nothing else.\n\n"
    user_prompt += python_code
    return user_prompt


In [26]:
def messages_for(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [31]:
def documentation_gpt(python):
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```python\n','').replace('```','')

In [48]:
def documentation_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_prompt,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```cpp\n','').replace('```','')

In [29]:
simple_function = """
def calculate_area(length, width):
    return length * width
"""

In [49]:
def document(python, model):
    if model=="GPT":
        result = documentation_gpt(python)
    elif model=="Claude":
        result = documentation_claude(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far

In [53]:
with gr.Blocks() as ui:
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
    with gr.Row():
        python = gr.Code(label="Code snippets:", lines=10, value=simple_function, language="python")
        python_doc = gr.Code(label="Documented code:", language="python")
    with gr.Row():
        convert = gr.Button("Document code")
    
    convert.click(document, inputs=[python, model], outputs=[python_doc])
    
ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.
